<h1 align="center"> Datos Complementarios </h1>

<center>En este Jupyter Notebook realizaremos Web Scraping a la API de <strong>Buenos Aires BA DATA</strong> el cual podemos consultar el Barrio, Comuna, Comisaría, Área Hospitalaria, Región Sanitaria y Distrito Escolar para un lugar determinado, através de coordenadas como <em>Latitud y Longitud</em>. En este caso, consultaremos el Barrio para poder complementar al dataset que tenemos de <strong>HOMICIDIOS</strong> en <em>Siniestros viales</em>.
<br>Y además, vamos a extraer información de los <strong>Censos del 2010 y 2022</strong> desde una página de <em>Wikipedia</em> para poder elaborar uno de los KPIs solicitados para este proyecto.</center>

## Importación de librerias

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

## API Buenos Aires BA DATA

### Lectura del dataset *Homicidios*

Leemos el archivo CSV y lo convertimo en un DataFrame.

In [2]:
homicidios = pd.read_csv('../Datasets/Cleans/homicidios_hechos.csv')

In [3]:
homicidios.head()

,Unnamed: 0,Id Hecho,Nro de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,...,Comuna,XY (CABA),POS X,POS Y,Participantes,Víctima,Acusado,Día de la Semana,Tipo de Día,Barrio
0,0,2016-0001,1,2016-01-01,2016,Enero,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,Viernes,Día de Semana,Villa Riachuelo
1,1,2016-0002,1,2016-01-02,2016,Enero,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Fin de Semana,Sobre Gral. Paz
2,2,2016-0003,1,2016-01-03,2016,Enero,3,07:00:00,7.0,AV ENTRE RIOS 2034,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,Domingo,Fin de Semana,Constitucion
3,3,2016-0004,1,2016-01-10,2016,Enero,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,...,8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,Domingo,Fin de Semana,Villa Lugano
4,4,2016-0005,1,2016-01-21,2016,Enero,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,...,1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,Día de Semana,Constitucion


### Creación de una función

Creamos una función para poder obtener a que Barrio pertenece de **CABA** apartir de los datos que tenemos de *Latitud* y *Longitud* en el DataFrame.

In [4]:
def obtener_barrio(latitud, longitud):
    """
    Obtiene el nombre del barrio correspondiente a las coordenadas geográficas dadas.

    Parameters:
    -----------
        latitud (float): La latitud de la ubicación.
        longitud (float): La longitud de la ubicación.

    Returns:
    --------
        str: El nombre del barrio si se encuentra, o "Desconocido" si no se encuentra o hay un error.
    """
    url = "https://datosabiertos-usig-apis.buenosaires.gob.ar/datos_utiles"
    params = {
        "x": longitud,
        "y": latitud,
        "formato": "json"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
        if "barrio" in data:
            return data["barrio"]
        else:
            return "Desconocido"
    except requests.exceptions.RequestException as e:
        print("Error al hacer la solicitud:", e)
        return "Desconocido"

### Aplicación de la función

Aplicamos la función creada en una nueva columna del DataFrame ***Homicidios***

In [ ]:
homicidios['Barrio'] = homicidios.apply(lambda row: obtener_barrio(row['POS Y'], row['POS X']), axis=1)

Corroboramos

In [ ]:
homicidios[['Barrio']]

,Barrio
0,Villa Riachuelo
1,
2,Constitucion
3,Villa Lugano
4,Constitucion
...,...
689,Parque Chacabuco
690,Parque Avellaneda
691,Flores
692,Parque Avellaneda


### Análisis de los datos

Analizamos los datos que obtuvimos de la API, ya que notamos que hay registros sin datos de Barrio.

In [ ]:
homicidios[homicidios.Barrio == '']

,Id Hecho,Nro de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Comuna,XY (CABA),POS X,POS Y,Participantes,Víctima,Acusado,Día de la Semana,Tipo de Día,Barrio
1,2016-0002,1,2016-01-02,2016,Enero,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Fin de Semana,
7,2016-0010,1,2016-01-29,2016,Enero,29,15:20:00,15.0,AV GENERAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,MOTO-AUTO,MOTO,AUTO,Viernes,Día de Semana,
17,2016-0025,1,2016-02-28,2016,Febrero,28,04:50:00,4.0,AV GRAL PAZ Y AV 27 DE FEBRERO,GRAL PAZ,...,8,Point (100077.95205948 91571.23307293),-58.462450,-34.705250,AUTO-AUTO,AUTO,AUTO,Domingo,Fin de Semana,
27,2016-0038,1,2016-03-21,2016,Marzo,21,08:06:00,8.0,AV GRAL PAZ COLECTORA Y ZUVIRIA,GRAL PAZ,...,8,Point (96926.79887463 94490.76318635),-58.496835,-34.678927,PEATON-PASAJEROS,PEATON,PASAJEROS,Lunes,Día de Semana,
32,2016-0045,1,2016-04-11,2016,Abril,11,10:25:00,10.0,AV GRAL PAZ Y AV DEL LIBERTADOR,GRAL PAZ,...,13,Point (99620.34936816 110483.29286598),-58.467435,-34.534769,MOTO-SD,MOTO,SD,Lunes,Día de Semana,
40,2016-0054,1,2016-04-23,2016,Abril,23,21:00:00,21.0,AU LUIS DELLEPIANE Y AV GRAL PAZ,AUTOPISTA,...,8,Point (97832.57143731 93489.83736533),-58.486954,-34.687952,MOTO-MOTO,MOTO,MOTO,Sábado,Fin de Semana,
51,2016-0074,1,2016-06-12,2016,Junio,12,06:40:00,6.0,AV GRAL PAZ Y RODOLFO SCAPINO,GRAL PAZ,...,8,Point (96731.44252651 94668.88351657),-58.498966,-34.677321,AUTO-AUTO,AUTO,AUTO,Domingo,Fin de Semana,
68,2016-0092,1,2016-07-15,2016,Julio,15,11:00:00,11.0,AV GRAL PAZ Y AV LIBERTADOR,GRAL PAZ,...,13,Point (99620.34936816 110483.29286598),-58.467435,-34.534769,MOTO-CARGAS,MOTO,CARGAS,Viernes,Día de Semana,
74,2016-0099,1,2016-07-29,2016,Julio,29,16:00:00,16.0,AV GRAL MOSCONI Y AV GRAL PAZ,GRAL PAZ,...,11,Point (94486.28631712 103468.59928582),-58.523406,-34.597988,CARGAS-CARGAS,CARGAS,CARGAS,Viernes,Día de Semana,
129,2016-0163,1,2016-12-10,2016,Diciembre,10,07:30:00,7.0,AV GRAL PAZ Y AV GENERAL MOSCONI,GRAL PAZ,...,11,Point (94486.28631712 103468.59928582),-58.523406,-34.597988,MOTO-AUTO,MOTO,AUTO,Sábado,Fin de Semana,


Observamos que hay muchos registros sin datos de Barrio, veamos cuantos.

In [ ]:
homicidios[homicidios.Barrio == ''].shape[0]

31

Hay 31 registro sin datos de Barrio, y notamos que el *Tipo de calle* de esos registros son **GRAL PAZ**. <br>
Veamos estos registros en un mapa

In [18]:
homicidios_sin_barrio = homicidios[homicidios['Barrio'] == '']

In [30]:
fig = px.scatter_mapbox(
    homicidios_sin_barrio,
    lat = 'POS Y',
    lon = 'POS X',
    zoom = 10.8,
    width = 800,   
    height = 680 
)

fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(title_x=0.5,
                  title_y=0.95, 
                  title_text='Siniestros sin datos de Barrio',
                  title_font=dict(size=24))
fig.update_traces(marker=dict(size=7, color='#113946'))  


fig.show()

Obseramos que estos siniestros ocurrieron sobre la Gral. Paz o al límite de la Ciudad Autonoma de Buenos Aires. Asi que procederemos a imputarlo como ***Sobre Gral Paz***  para dar a entender los datos.

In [40]:
homicidios['Barrio'] = homicidios['Barrio'].replace('', 'Sobre Gral. Paz')

### Guardado del DataFrame

In [42]:
homicidios.to_csv('../Datasets/Cleans/homicidios_hechos.csv', index=False, encoding='utf-8')

## Página de *Wikipedia*

La página que usaremos es esta: https://es.wikipedia.org/wiki/Censo_argentino_de_2022, el cual tiene información estadístico del Censo del 2010 y 2022 categorizado por *Jurisdicción* y *Aglomerado Urbano*.

In [2]:
url = 'https://es.wikipedia.org/wiki/Censo_argentino_de_2022'

# Realizamos una solicitud GET a la página
response = requests.get(url)

# Analizamos el contenido HTML de la página
soup = BeautifulSoup(response.content, 'html.parser')

# Buscamos la segunda tabla de la pagina con la clase 'wikitable'
tabla = soup.find_all('table', class_='wikitable')[0]

# Extraemos los datos de la tabla y lo guardamos en una lista
datos = []
filas = tabla.find_all('tr')

for fila in filas[1:]:  
    celdas = fila.find_all(['th', 'td'])
    fila_datos = [celda.get_text(strip=True) for celda in celdas]
    datos.append(fila_datos)

Vemos los datos extraidos y seleccionamos los datos de la Ciudad Autónoma de Buenos Aires.

In [4]:
for fila in datos:
    print(fila)

['1', 'Provincia de Buenos Aires', '15.625.084[6]\u200b', '17.569.053[7]\u200b', '12,4', '9,8']
['2', 'Córdoba', '3.308.876', '3.978.984', '20,3', '15,5']
['3', 'Santa Fe', '3.194.537', '3.556.522', '11,3', '9,0']
['4', 'Ciudad Autónoma de Buenos Aires', '2.890.151', '3.120.612', '8,0', '6,4']
['5', 'Mendoza', '1.738.929', '2.014.533', '15,8', '12,3']
['6', 'Tucumán', '1.448.188', '1.703.186', '17,6', '13,6']
['7', 'Salta', '1.214.441', '1.440.672', '18,6', '14,3']
['8', 'Entre Ríos', '1.235.994', '1.426.426', '15,4', '12,0']
['9', 'Misiones', '1.101.593', '1.280.960', '16,3', '12,7']
['10', 'Corrientes', '992.595', '1.197.553', '20,6', '15,8']
['11', 'Chaco', '1.055.259', '1.142.963', '8,3', '6,7']
['12', 'Santiago del Estero', '874.006', '1.054.028', '20,6', '15,7']
['13', 'San Juan', '681.055', '818.234', '20,1', '15,4']
['14', 'Jujuy', '673.307', '797.955', '18,5', '14,3']
['15', 'Río Negro', '638.645', '762.067', '19,3', '14,8']
['16', 'Neuquén', '551.266', '726.590', '31,8', '23,

In [5]:
data_caba = datos[3]

In [6]:
data_caba

['4',
 'Ciudad Autónoma de Buenos Aires',
 '2.890.151',
 '3.120.612',
 '8,0',
 '6,4']

Seleccionamos los valores de la población del 2010 y 2022 de la lista

In [7]:
poblacion_2010 = int(data_caba[2].replace('.', '').replace(',', ''))
poblacion_2022 = int(data_caba[3].replace('.', '').replace(',', ''))

Creamos una lista de población por año calculando el crecimiento anual promedio.

In [9]:
anios = list(range(2010, 2023))

poblacion_por_anio = [poblacion_2010 + (poblacion_2022 - poblacion_2010) * (anio - 2010) / 12 for anio in anios]

poblacion_por_anio = list(map(int, poblacion_por_anio))

In [10]:
poblacion_por_anio

[2890151,
 2909356,
 2928561,
 2947766,
 2966971,
 2986176,
 3005381,
 3024586,
 3043791,
 3062996,
 3082201,
 3101406,
 3120612]

Creamos un DataFrame con los datos

In [11]:
data = {'Año': anios, 'Población': poblacion_por_anio}
poblacion_caba = pd.DataFrame(data)

In [12]:
poblacion_caba

,Año,Población
0,2010,2890151
1,2011,2909356
2,2012,2928561
3,2013,2947766
4,2014,2966971
5,2015,2986176
6,2016,3005381
7,2017,3024586
8,2018,3043791
9,2019,3062996


Guardamos el DataFrame en un archivo CSV

In [13]:
poblacion_caba.to_csv('../Datasets/Cleans/población_caba.csv', index= False, encoding='utf-8')